`ore-psych-ml-ibs-loocv.ipynb` notebook (ver 2022-12-16) for the Leave-One-Out Cross-Validtaion (LOOCV) procedure accompanying the manuscript 

### A psychological symptom based machine learning model for clinical evaluation of irritable bowel syndrome
 
by Noman Haleem, ..., Arvid Lundervold to be submitted to Open Research Europe.


Runs under the following conda environment, denoted `oreibs0`:

```
conda create --name oreibs0 -c conda-forge scikit-learn python=3.8
conda activate oreibs0
conda install ipython
conda install jupyter
conda install pandas
conda install matplotlib
conda install statsmodels
conda install -c conda-forge  pingouin
conda install -c plotly plotly=5.10.0
```


In [1]:
%matplotlib inline

### Import libraries 

In [2]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedShuffleSplit

### Read the `testing`, `training` and `validation` .csv files generatet previously

In [3]:
df_test = pd.read_csv('data/testing.csv')
df_train = pd.read_csv('data/training.csv')
df_validation = pd.read_csv('data/validation.csv')

### Contatenate these dataframes to prepare the full dataset  for LOOCV

In [4]:
df = pd.concat([df_test, df_train, df_validation])  #.drop_duplicates(keep=False)

#### Inspect

In [5]:
df.shape

(84, 39)

In [6]:
print(df.columns)

Index(['pt_anon_ids', 'Category', 'Sub_category', 'Gender', 'Age', 'BIS_Q1_BL',
       'BIS_Q2_BL', 'BIS_Q3_BL', 'BIS_Q4_BL', 'BIS_Q5_BL', 'BIS_Q6_BL',
       'FSS_Q1_BL', 'FSS_Q10_BL', 'FSS_Q11_BL', 'FSS_Q12_BL', 'FSS_Q13_BL',
       'FSS_Q2_BL', 'FSS_Q3_BL', 'FSS_Q4_BL', 'FSS_Q5_BL', 'FSS_Q6_BL',
       'FSS_Q7_BL', 'FSS_Q8_BL', 'FSS_Q9_BL', 'HADS_q1_BL', 'HADS_q2_BL',
       'HADS_q3_BL', 'HADS_q4_BL', 'HADS_q5_BL', 'HADS_q6_BL', 'HADS_q7_BL',
       'HADS_q8_BL', 'HADS_q9_BL', 'HADS_q10_BL', 'HADS_q11_BL', 'HADS_q12_BL',
       'HADS_q13_BL', 'HADS_q14_BL', 'stratification'],
      dtype='object')


In [7]:
df.head()

,pt_anon_ids,Category,Sub_category,Gender,Age,BIS_Q1_BL,BIS_Q2_BL,BIS_Q3_BL,BIS_Q4_BL,BIS_Q5_BL,...,HADS_q6_BL,HADS_q7_BL,HADS_q8_BL,HADS_q9_BL,HADS_q10_BL,HADS_q11_BL,HADS_q12_BL,HADS_q13_BL,HADS_q14_BL,stratification
0,BGA_082,HC,HC,F,46.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HC_F
1,BGA_136,IBS,Diarrhea,F,30.0,7.0,3.0,3.0,5.0,3.0,...,1.0,2.0,1.0,2.0,1.0,3.0,2.0,3.0,0.0,Diarrhea_F
2,BGA_095,IBS,Mixed,M,28.0,3.0,0.0,0.0,6.0,3.0,...,1.0,2.0,1.0,2.0,0.0,1.0,2.0,2.0,0.0,Mixed_M
3,BGA_153,IBS,Mixed,F,32.0,5.0,2.0,2.0,5.0,3.0,...,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,Mixed_F
4,BGA_154,IBS,Mixed,F,37.0,0.0,0.0,0.0,7.0,5.0,...,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,Mixed_F


In [8]:
df.tail()

,pt_anon_ids,Category,Sub_category,Gender,Age,BIS_Q1_BL,BIS_Q2_BL,BIS_Q3_BL,BIS_Q4_BL,BIS_Q5_BL,...,HADS_q6_BL,HADS_q7_BL,HADS_q8_BL,HADS_q9_BL,HADS_q10_BL,HADS_q11_BL,HADS_q12_BL,HADS_q13_BL,HADS_q14_BL,stratification
12,BGA_157,HC,HC,F,57.0,7.0,7.0,2.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HC_F
13,BGA_119,HC,HC,F,36.0,2.0,0.0,1.0,4.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,HC_F
14,BGA_104,IBS,Diarrhea,M,30.0,6.0,0.0,0.0,5.0,1.0,...,2.0,2.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,Diarrhea_M
15,BGA_099,HC,HC,M,39.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,HC_M
16,BGA_061,IBS,Constipation,F,42.0,5.0,2.0,4.0,4.0,3.0,...,1.0,2.0,2.0,2.0,0.0,1.0,2.0,2.0,1.0,Constipation_F


### Make the X and the y in the framework $$y \sim f(X, \theta)$$

where $f$ is the model (LR, RF, SVM, etc.) and $\theta$ is parameters of the model being learned.

In [9]:
# The selected variables by Noman
Xlist = ['Gender', 'HADS_q1_BL', 'HADS_q3_BL', 'HADS_q5_BL',
         'FSS_Q1_BL', 'FSS_Q5_BL', 'FSS_Q6_BL', 'FSS_Q8_BL', 'FSS_Q12_BL']

dfX1 = df[Xlist]
dfX1

,Gender,HADS_q1_BL,HADS_q3_BL,HADS_q5_BL,FSS_Q1_BL,FSS_Q5_BL,FSS_Q6_BL,FSS_Q8_BL,FSS_Q12_BL
0,F,0.0,0.0,0.0,3.0,3.0,1.0,3.0,2.0
1,F,3.0,2.0,2.0,4.0,4.0,1.0,3.0,3.0
2,M,2.0,2.0,2.0,5.0,5.0,1.0,3.0,5.0
3,F,2.0,0.0,1.0,4.0,4.0,4.0,4.0,3.0
4,F,1.0,1.0,0.0,5.0,5.0,5.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...
12,F,1.0,0.0,0.0,3.0,3.0,1.0,3.0,1.0
13,F,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0
14,M,2.0,1.0,2.0,4.0,3.0,3.0,3.0,3.0
15,M,1.0,0.0,1.0,3.0,3.0,3.0,3.0,1.0


In [10]:
def numberfygender(str):
    if str == 'F':
        return 0
    elif str == 'M':
        return 1
    else:
        return 99

In [11]:
dfX = dfX1.copy()
gendernum_list = list(map(numberfygender, dfX1['Gender']))
dfX['Gender'] = gendernum_list

In [12]:
# Make numpy ndarray
X = dfX.to_numpy()

In [13]:
dfy = df['Category']

In [14]:
dfXy = pd.concat([dfX, dfy], axis=1)

In [15]:
# Reset index
dfXy = dfXy.reset_index(drop=True)
dfXy

,Gender,HADS_q1_BL,HADS_q3_BL,HADS_q5_BL,FSS_Q1_BL,FSS_Q5_BL,FSS_Q6_BL,FSS_Q8_BL,FSS_Q12_BL,Category
0,0,0.0,0.0,0.0,3.0,3.0,1.0,3.0,2.0,HC
1,0,3.0,2.0,2.0,4.0,4.0,1.0,3.0,3.0,IBS
2,1,2.0,2.0,2.0,5.0,5.0,1.0,3.0,5.0,IBS
3,0,2.0,0.0,1.0,4.0,4.0,4.0,4.0,3.0,IBS
4,0,1.0,1.0,0.0,5.0,5.0,5.0,4.0,5.0,IBS
...,...,...,...,...,...,...,...,...,...,...
79,0,1.0,0.0,0.0,3.0,3.0,1.0,3.0,1.0,HC
80,0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,HC
81,1,2.0,1.0,2.0,4.0,3.0,3.0,3.0,3.0,IBS
82,1,1.0,0.0,1.0,3.0,3.0,3.0,3.0,1.0,HC


In [16]:
y = dfy.to_numpy()
y

array(['HC', 'IBS', 'IBS', 'IBS', 'IBS', 'HC', 'HC', 'IBS', 'IBS', 'HC',
       'IBS', 'IBS', 'HC', 'IBS', 'IBS', 'HC', 'HC', 'IBS', 'IBS', 'IBS',
       'IBS', 'IBS', 'IBS', 'IBS', 'IBS', 'IBS', 'IBS', 'IBS', 'HC', 'HC',
       'IBS', 'HC', 'IBS', 'HC', 'HC', 'IBS', 'HC', 'IBS', 'HC', 'IBS',
       'IBS', 'HC', 'HC', 'HC', 'HC', 'HC', 'HC', 'IBS', 'IBS', 'HC',
       'HC', 'IBS', 'IBS', 'HC', 'HC', 'HC', 'HC', 'IBS', 'IBS', 'IBS',
       'IBS', 'IBS', 'IBS', 'HC', 'IBS', 'HC', 'IBS', 'HC', 'IBS', 'IBS',
       'HC', 'IBS', 'IBS', 'IBS', 'HC', 'IBS', 'IBS', 'HC', 'IBS', 'HC',
       'HC', 'IBS', 'HC', 'IBS'], dtype=object)

###  Perform the LOOCV with different classfiers

https://machinelearningmastery.com/loocv-for-evaluating-machine-learning-algorithms/

The **Leave-One-Out cross-validator** (`scikit-learn`)

Provides train/test indices to split data in train/test sets. Each sample is used once as a test set (singleton) while the remaining samples form the training set.

When compared with $k$-fold cross validation, one builds $n$  models from  $n$ samples instead of $k$ models, 
where  $n > k$. Moreover, each is trained on $n-1$ samples rather than $(k-1)n/k$. In both ways, assuming $k$ is not too large and $k < n$ , LOO is more computationally expensive than $k$-fold cross validation.

In terms of accuracy, LOO often results in high variance as an estimator for the test error. Intuitively, since $n-1$ of the $n$ samples are used to build each model, models constructed from folds are virtually identical to each other and to the model built from the entire training set.

Note: `LeaveOneOut()` is equivalent to `KFold(n_splits=n)` and `LeavePOut(p=1)` where `n` is the number of samples.

Due to the high number of test sets (which is the same as the number of samples) this cross-validation method can be very costly. For large datasets one should favor `KFold`, `ShuffleSplit` or `StratifiedKFold`.


**References:**

- http://www.faqs.org/faqs/ai-faq/neural-nets/part3/section-12.html;

- T. Hastie, R. Tibshirani, J. Friedman, The Elements of Statistical Learning, Springer 2009

- L. Breiman, P. Spector Submodel selection and evaluation in regression: The X-random case, International Statistical Review 1992;

- R. Kohavi, A Study of Cross-Validation and Bootstrap for Accuracy Estimation and Model Selection, Intl. Jnt. Conf. AI

- R. Bharat Rao, G. Fung, R. Rosales, On the Dangers of Cross-Validation. An Experimental Evaluation, SIAM 2008;

- G. James, D. Witten, T. Hastie, R Tibshirani, An Introduction to Statistical Learning, Springer 2013.


See more at https://scikit-learn.org/stable/modules/cross_validation.html#leave-one-out

#### _We are making the following very transparent and explict du to the small sample size ..._

In [17]:
%%time
# create loocv procedure
cv = LeaveOneOut()

# enumerate splits
y_true, y_predRF, y_predLR, y_predSVC = list(), list(), list(), list()

k=0
for train_ix, test_ix in cv.split(X):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # fit models
    modelLR = LogisticRegression(random_state=1, max_iter=1000)
    modelLR.fit(X_train, y_train)
    
    modelRF = RandomForestClassifier(random_state=1)
    modelRF.fit(X_train, y_train)
    
    modelSVC = SVC(random_state=1)
    modelSVC.fit(X_train, y_train)
    
    # evaluate model
    yhatLR = modelLR.predict(X_test)
    yhatRF = modelRF.predict(X_test)
    yhatSVC = modelSVC.predict(X_test)
    
    print('y_true, yhatLR, yhatRF, yhatSVC (k=%d):' % k, f"{y_test[0]}_", yhatLR[0], yhatRF[0], yhatSVC[0])
    # store
    y_true.append(y_test[0])
    y_predLR.append(yhatLR[0])
    y_predRF.append(yhatRF[0])
    y_predSVC.append(yhatSVC[0])
    
    k=k+1
# calculate accuracy
accLR = accuracy_score(y_true, y_predLR)
accRF = accuracy_score(y_true, y_predRF)
accSVC = accuracy_score(y_true, y_predSVC)

baccLR = balanced_accuracy_score(y_true, y_predLR)
baccRF = balanced_accuracy_score(y_true, y_predRF)
baccSVC = balanced_accuracy_score(y_true, y_predSVC)

print('Accuracy LR: %.3f' % accLR)
print('Accuracy RF: %.3f' % accRF)
print('Accuracy SVC: %.3f' % accSVC)

print('Balanced accuracy LR: %.3f' % baccLR)
print('Balanced accuracy RF: %.3f' % baccRF)
print('Balanced accuracy SVC: %.3f' % baccSVC)

y_true, yhatLR, yhatRF, yhatSVC (k=0): HC_ HC HC HC
y_true, yhatLR, yhatRF, yhatSVC (k=1): IBS_ IBS HC IBS
y_true, yhatLR, yhatRF, yhatSVC (k=2): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=3): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=4): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=5): HC_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=6): HC_ HC HC HC
y_true, yhatLR, yhatRF, yhatSVC (k=7): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=8): IBS_ IBS HC HC
y_true, yhatLR, yhatRF, yhatSVC (k=9): HC_ HC HC HC
y_true, yhatLR, yhatRF, yhatSVC (k=10): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=11): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=12): HC_ HC HC HC
y_true, yhatLR, yhatRF, yhatSVC (k=13): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=14): IBS_ IBS IBS IBS
y_true, yhatLR, yhatRF, yhatSVC (k=15): HC_ HC HC HC
y_true, yhatLR, yhatRF, yhatSVC (k=16): HC_ HC IBS HC
y_true, yhatLR, yhatRF, yhatSVC (k=17): IBS_ IBS IBS IBS
y_true, yha

In [18]:
cat = ['HC','IBS']
pd.Series(cat).astype('category').cat.codes.values

array([0, 1], dtype=int8)

In [19]:
def numberfycat(str):
    if str == 'HC':
        return 0
    elif str == 'IBS':
        return 1
    else:
        return 99

In [20]:
num_y_true = list(map(numberfycat, y_true))
num_y_predLR = list(map(numberfycat, y_predLR))
num_y_predRF = list(map(numberfycat, y_predRF))
num_y_predSVC = list(map(numberfycat, y_predSVC))

In [21]:
# calculate performance metrics
accLR = accuracy_score(num_y_true, num_y_predLR)
accRF = accuracy_score(num_y_true, num_y_predRF)
accSVC = accuracy_score(num_y_true, num_y_predSVC)

baccLR = balanced_accuracy_score(num_y_true, num_y_predLR)
baccRF = balanced_accuracy_score(num_y_true, num_y_predRF)
baccSVC = balanced_accuracy_score(num_y_true, num_y_predSVC)

f1LR = f1_score(num_y_true, num_y_predLR)
f1RF = f1_score(num_y_true, num_y_predRF)
f1SVC = f1_score(num_y_true, num_y_predSVC)
                
precLR = precision_score(num_y_true, num_y_predLR)
precRF = precision_score(num_y_true, num_y_predRF)
precSVC = precision_score(num_y_true, num_y_predSVC)

recallLR = recall_score(num_y_true, num_y_predLR)
recallRF = recall_score(num_y_true, num_y_predRF)
recallSVC = recall_score(num_y_true, num_y_predSVC)

print('Accuracy LR: %.3f' % accLR)
print('Accuracy RF: %.3f' % accRF)
print('Accuracy SVC: %.3f' % accSVC)

print('Balanced accuracy LR: %.3f' % baccLR)
print('Balanced accuracy RF: %.3f' % baccRF)
print('Balanced accuracy SVC: %.3f' % baccSVC)

print('F1-score LR: %.3f' % f1LR)
print('F1-score RF: %.3f' % f1RF)
print('F1-score SVC: %.3f' % f1SVC)

print('Precision LR: %.3f' % precLR)
print('Precision RF: %.3f' % precRF)
print('Precision SVC: %.3f' % precSVC)

print('Recall LR: %.3f' % recallLR)
print('Recall RF: %.3f' % recallRF)
print('Recall SVC: %.3f' % recallSVC)


Accuracy LR: 0.845
Accuracy RF: 0.786
Accuracy SVC: 0.821
Balanced accuracy LR: 0.839
Balanced accuracy RF: 0.780
Balanced accuracy SVC: 0.814
F1-score LR: 0.869
F1-score RF: 0.816
F1-score SVC: 0.848
Precision LR: 0.860
Precision RF: 0.816
Precision SVC: 0.840
Recall LR: 0.878
Recall RF: 0.816
Recall SVC: 0.857
